In [1]:
! pip install -q  "groq==0.13.0" "autogen-agentchat==0.2.39"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.2/382.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00


In [2]:
import os
import autogen
from autogen import AssistantAgent, ConversableAgent
from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
api_key = userdata.get('GROQ_API_KEY')
model = "llama-3.3-70b-versatile"

In [4]:
config_list = [
    {
    "model": model,
    "api_key": api_key,
    "api_type": "groq",
    }
]

In [5]:
llm_config = {
    "timeout": 120,
    "cache_seed": 41,
    "config_list": config_list,
    "temperature": 0,
}

In `AssistantAgent`, you must provide llm_config

In [6]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes.
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """,
)

scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=llm_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code.""",
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
    The plan may involve an engineer who can write code. The executed by executor.
    Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a executor.
    """,
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer. If it fails try again with the fix. Finally report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)



In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, planner, scientist, executor, critic], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

response = user_proxy.initiate_chat(
    manager,
    message="""
find 5 papers on LLM applications from arxiv in the last week, create a markdown table of different domains. Generate code and make the executor run it to find the information.
""",
)
print("Response:")
response

Admin (to chat_manager):


find 5 papers on LLM applications from arxiv in the last week, create a markdown table of different domains. Generate code and make the executor run it to find the information.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

**Plan: Retrieving Recent LLM Application Papers from arXiv and Creating a Markdown Table**

### Step 1: Define Requirements and Search Query (Planner)
- Identify the search query to find relevant papers on LLM applications.
- Determine the time frame for the search (last week).
- Decide on the structure of the markdown table for organizing the results.

### Step 2: Engineer Task - Write Code to Retrieve Papers and Generate Markdown Table
- **Engineer's Task**: Write a Python script using the `arxiv` library to search for papers related to LLM applications published in the last week.
- The script should:
  1. Query arXiv for papers matching the LLM appl

ChatResult(chat_id=None, chat_history=[{'content': '\nfind 5 papers on LLM applications from arxiv in the last week, create a markdown table of different domains. Generate code and make the executor run it to find the information.\n', 'role': 'assistant', 'name': 'Admin'}, {'content': '**Plan: Retrieving Recent LLM Application Papers from arXiv and Creating a Markdown Table**\n\n### Step 1: Define Requirements and Search Query (Planner)\n- Identify the search query to find relevant papers on LLM applications.\n- Determine the time frame for the search (last week).\n- Decide on the structure of the markdown table for organizing the results.\n\n### Step 2: Engineer Task - Write Code to Retrieve Papers and Generate Markdown Table\n- **Engineer\'s Task**: Write a Python script using the `arxiv` library to search for papers related to LLM applications published in the last week.\n- The script should:\n  1. Query arXiv for papers matching the LLM application search query within the specified